In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("./heart.csv")
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,59,1,1,140,221,0,1,164,1,0.0,2,0,2,1
1021,60,1,0,125,258,0,0,141,1,2.8,1,1,3,0
1022,47,1,0,110,275,0,0,118,1,1.0,1,1,2,0
1023,50,0,0,110,254,0,0,159,0,0.0,2,0,2,1


In [6]:
df.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')

In [ ]:
# from sklearn.preprocessing import LabelEncoder

# columns_to_encode = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']

# label_encoder = LabelEncoder()

# for column in columns_to_encode:
#     if column in df.columns:
#         df[column] = label_encoder.fit_transform(df[column])

In [7]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,59,1,1,140,221,0,1,164,1,0.0,2,0,2,1
1021,60,1,0,125,258,0,0,141,1,2.8,1,1,3,0
1022,47,1,0,110,275,0,0,118,1,1.0,1,1,2,0
1023,50,0,0,110,254,0,0,159,0,0.0,2,0,2,1


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

y = df['target']
X = df.drop('target', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', save_best_only=True, verbose=1)

model.fit(X_train, y_train, epochs=2000, batch_size=64, validation_split=0.1, callbacks=[checkpoint], verbose=1)

best_model = tf.keras.models.load_model('best_model.h5')

Epoch 1/2000


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


 1/12 ━━━━━━━━━━━━━━━━━━━━ 37s 3s/step - accuracy: 0.4844 - loss: 7.8440
Epoch 1: val_accuracy improved from -inf to 0.46341, saving model to best_model.h5


12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - accuracy: 0.4938 - loss: 4.3523 - val_accuracy: 0.4634 - val_loss: 0.6876
Epoch 2/2000
 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.3750 - loss: 2.5990
Epoch 2: val_accuracy improved from 0.46341 to 0.57317, saving model to best_model.h5


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4683 - loss: 1.8863 - val_accuracy: 0.5732 - val_loss: 0.7052
Epoch 3/2000
 1/12 ━━━━━━━━━━━━━━━━━━━━ 1s 153ms/step - accuracy: 0.5312 - loss: 1.0941
Epoch 3: val_accuracy did not improve from 0.57317
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4971 - loss: 1.2031 - val_accuracy: 0.5000 - val_loss: 0.6974
Epoch 4/2000
 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.4375 - loss: 0.9991
Epoch 4: val_accuracy did not improve from 0.57317
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5024 - loss: 0.9636 - val_accuracy: 0.5122 - val_loss: 0.6935
Epoch 5/2000
 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.5156 - loss: 0.8707
Epoch 5: val_accuracy did not improve from 0.57317
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5211 - loss: 0.8348 - val_accuracy: 0.5122 - val_loss: 0.6931
Epoch 6/2000
 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.3906 - loss: 0.8827
Epoch 6: val_accuracy improved fro

...

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9169 - loss: 0.2007 - val_accuracy: 0.9146 - val_loss: 0.2683
Epoch 569/2000
 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.9219 - loss: 0.1363
Epoch 569: val_accuracy did not improve from 0.91463
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9344 - loss: 0.1574 - val_accuracy: 0.8659 - val_loss: 0.2409
Epoch 570/2000
 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.9375 - loss: 0.1753
Epoch 570: val_accuracy did not improve from 0.91463
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9339 - loss: 0.1713 - val_accuracy: 0.9146 - val_loss: 0.2275
Epoch 571/2000
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9360 - loss: 0.1719 
Epoch 571: val_accuracy did not improve from 0.91463
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9355 - loss: 0.1715 - val_accuracy: 0.8780 - val_loss: 0.2569
Epoch 572/2000
 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9531 - loss: 0.1118
Epoch 572: val_accura

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9430 - loss: 0.1559 - val_accuracy: 0.9268 - val_loss: 0.2210
Epoch 624/2000
 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.9375 - loss: 0.1937
Epoch 624: val_accuracy did not improve from 0.92683
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9543 - loss: 0.1502 - val_accuracy: 0.9024 - val_loss: 0.2598
Epoch 625/2000
 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.9531 - loss: 0.1387
Epoch 625: val_accuracy did not improve from 0.92683
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9528 - loss: 0.1316 - val_accuracy: 0.9268 - val_loss: 0.2048
Epoch 626/2000
 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.9688 - loss: 0.0826
Epoch 626: val_accuracy did not improve from 0.92683
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9380 - loss: 0.1448 - val_accuracy: 0.9024 - val_loss: 0.2352
Epoch 627/2000
 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.9688 - loss: 0.0903
Epoch 627: val_accura

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9317 - loss: 0.1589 - val_accuracy: 0.9390 - val_loss: 0.2168
Epoch 640/2000
 1/12 ━━━━━━━━━━━━━━━━━━━━ 1s 118ms/step - accuracy: 0.9375 - loss: 0.1448
Epoch 640: val_accuracy did not improve from 0.93902
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9373 - loss: 0.1575 - val_accuracy: 0.9024 - val_loss: 0.2555
Epoch 641/2000
 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9062 - loss: 0.1889
Epoch 641: val_accuracy did not improve from 0.93902
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9392 - loss: 0.1571 - val_accuracy: 0.9024 - val_loss: 0.2235
Epoch 642/2000
 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.9219 - loss: 0.1857
Epoch 642: val_accuracy did not improve from 0.93902
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9360 - loss: 0.1562 - val_accuracy: 0.9390 - val_loss: 0.2123
Epoch 643/2000
 1/12 ━━━━━━━━━━━━━━━━━━━━ 1s 163ms/step - accuracy: 0.9688 - loss: 0.0768
Epoch 643: val_accu

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9454 - loss: 0.1384 - val_accuracy: 0.9512 - val_loss: 0.1802
Epoch 705/2000
 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.9688 - loss: 0.1071
Epoch 705: val_accuracy did not improve from 0.95122
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9491 - loss: 0.1379 - val_accuracy: 0.9146 - val_loss: 0.2330
Epoch 706/2000
 1/12 ━━━━━━━━━━━━━━━━━━━━ 1s 171ms/step - accuracy: 0.9688 - loss: 0.1075
Epoch 706: val_accuracy did not improve from 0.95122
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9365 - loss: 0.1737 - val_accuracy: 0.8659 - val_loss: 0.2856
Epoch 707/2000
 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.9062 - loss: 0.1956
Epoch 707: val_accuracy did not improve from 0.95122
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9252 - loss: 0.1867 - val_accuracy: 0.9146 - val_loss: 0.2251
Epoch 708/2000
 1/12 ━━━━━━━━━━━━━━━━━━━━ 1s 176ms/step - accuracy: 0.9531 - loss: 0.1008
Epoch 708: val_accu

Streaming output truncated to the last 5000 lines.
Epoch 751/2000
 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.9375 - loss: 0.2358
Epoch 751: val_accuracy did not improve from 0.97561
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9328 - loss: 0.1838 - val_accuracy: 0.9268 - val_loss: 0.2361
Epoch 752/2000
 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.9375 - loss: 0.1738
Epoch 752: val_accuracy did not improve from 0.97561
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9495 - loss: 0.1616 - val_accuracy: 0.9512 - val_loss: 0.1593
Epoch 753/2000
 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.9375 - loss: 0.1787
Epoch 753: val_accuracy did not improve from 0.97561
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9363 - loss: 0.1484 - val_accuracy: 0.8780 - val_loss: 0.2488
Epoch 754/2000
 1/12 ━━━━━━━━━━━━━━━━━━━━ 1s 140ms/step - accuracy: 0.8906 - loss: 0.2286
Epoch 754: val_accuracy did not improve from 0.97561
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12

In [ ]:
best_model = tf.keras.models.load_model('best_model.h5')

In [ ]:
probabilities = best_model.predict(X_test)

threshold = 0.5

binary_predictions = np.where(probabilities > threshold, 1, 0)


print("Classification Report:")
print(classification_report(y_test, binary_predictions))

7/7 [==============================] - 0s 2ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.96      0.96       102
           1       0.96      0.95      0.96       103

    accuracy                           0.96       205
   macro avg       0.96      0.96      0.96       205
weighted avg       0.96      0.96      0.96       205



In [ ]:
print(binary_predictions)

[[1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]


In [ ]:
print(y_test)

527    1
359    1
447    0
31     1
621    0
      ..
832    1
796    1
644    1
404    0
842    0
Name: target, Length: 205, dtype: int64
